In [1]:
!pip install opencv-python
!pip install imutils
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import tensorflow as tf
from PIL import Image
from pathlib import Path

Traceback (most recent call last):
  File "c:\users\dell\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\dell\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\DELL\Anaconda3\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
Traceback (most recent call last):
  File "c:\users\dell\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\dell\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\DELL\Anaconda3\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


ImportError: cannot import name 'RMSprop' from 'keras.optimizers' (C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\keras\optimizers.py)

In [ ]:
#building neural network
model = Sequential([Conv2D(100,(3,3), activation='relu', input_shape=(150,150,3))
                    ,MaxPooling2D(2,2),
                    Conv2D(100,(3,3),activation = 'relu'),
                    MaxPooling2D(2,2), Flatten(),Dropout(0.5),
                    Dense(50, activation = 'relu'),
                    Dense(2, activation = 'softmax')])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#image generation and augmentation
TRAINING_DIR = r"C:\Users\DELL\Downloads\face-mask-dataset"
train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range = 40, width_shift_range = 0.2, height_shift_range=0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size = 10,
                                                    target_size=(150,150))

VALIDATION_DIR = r"E:\DSC"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size = 10, target_size=(150,150))


In [ ]:
#Initializing a callback checkpoint to keep saving best model after each epoch while training
checkpoint = ModelCheckpoint(r'C:\Users\DELL\Downloads\face-mask-dataset',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [22]:
#for getting unidentified images as they cause errors in the code
bad_list=[]
dir=r'C:\Users\DELL\Downloads\face-mask-dataset'
subdir_list=os.listdir(dir) # create a list of the sub directories in the directory ie train or test
for d in subdir_list:  # iterate through the sub directories train and test
    dpath=os.path.join (dir, d) # create path to sub directory
    if d in ['test', 'train']:
        class_list=os.listdir(dpath) # list of classes ie dog or cat
       # print (class_list)
        for klass in class_list: # iterate through the two classes
            class_path=os.path.join(dpath, klass) # path to class directory
            #print(class_path)
            file_list=os.listdir(class_path) # create list of files in class directory
            for f in file_list: # iterate through the files
                fpath=os.path.join (class_path,f)
                index=f.rfind('.') # find index of period infilename
                ext=f[index+1:] # get the files extension
                if ext  not in ['jpg', 'png', 'bmp', 'gif']:
                    print(f'file {fpath}  has an invalid extension {ext}')
                    bad_list.append(fpath)                    
                else:
                    try:
                        img=cv2.imread(fpath)
                        size=img.shape
                    except:
                        print(f'file {fpath} is not a valid image file ')
                        bad_list.append(fpath)
                       
print (bad_list)

[]


In [24]:
folder_path = r'C:\Users\DELL\Downloads\face-mask-dataset\train'
extensions = []
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])

            


In [25]:
#Training the model
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator, callbacks = [checkpoint])
tf.keras.layers.Dense(10, activation='softmax')

Epoch 1/10
129/129 [==============================] - ETA: 0s - loss: 0.8189 - accuracy: 0.0792WARNING:tensorflow:From C:\Users\DELL\anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\DELL\Downloads\face-mask-dataset\assets
129/129 [==============================] - 53s 415ms/step - loss: 0.8189 - accuracy: 0.0792 - val_loss: 0.6931 - val_accuracy: 0.0000e+00
Epoch 2/10
129/129 [==============================] - 49s 381ms/step - loss: 0.6931 - accuracy: 0.0000e+00 - val_loss: 0.6931 - val_accuracy: 0.0000e+00
Epoch 3/10
129/129 [==============================] - 5

In [32]:
#I actually re-wrote a code I found in https://data-flair.training/blogs/face-mask-detection-with-python/ 
#and then fixed the errors that popped out